# Применение линейных методов классификации для диагностики кризиса теплообмена в ЯЭУ.
Группа: ИВТ-М20.<br/>
Студент: Лискунов Роман Геннадьвич

In [1]:
from typing import Union, Any

from pandas import (
    DataFrame,
    read_csv
)
from sklearn.metrics import (
    accuracy_score,
    classification_report
)
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


## 1. Прочтите данные из файлов

In [2]:
transformators: DataFrame = DataFrame(
    read_csv(
        'data/transformators.csv',
        sep=',',
        header=0
    )
)

In [3]:
x = transformators.drop('label', axis=1)
y = transformators.label

## 2. Отобразите несколько первых и несколько последних записей.

In [4]:
print(transformators.head())
print('\n')
print(transformators.tail())

         H2        CO      C2H4      C2H2  label
0  0.002590  0.016491  0.002876  0.000336      1
1  0.002825  0.014728  0.001992  0.000339      1
2  0.002764  0.017030  0.003394  0.000195      1
3  0.000508  0.016334  0.004104  0.000380      1
4  0.002654  0.021278  0.004068  0.000124      1


            H2        CO      C2H4      C2H2  label
2095  0.002549  0.009460  0.007465  0.000242      1
2096  0.000915  0.005770  0.007230  0.000081      3
2097  0.001565  0.002492  0.008529  0.000176      3
2098  0.001688  0.020291  0.007305  0.000252      1
2099  0.002935  0.028001  0.008048  0.000219      1


## 3. Разбейте данные для классификации

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, train_size=0.7)

##  4. Примените метод опорных векторов для классификации (SVC)

In [6]:
clf: SVC = SVC()
clf.fit(x, y)

SVC()

## 5. Сделайте предсказание на тестовой выборке

In [7]:
pred = clf.predict(x_test)
print(f"prediction: {pred}", end='\n\n')
print(f"accuracy_score: {round(accuracy_score(y_test, pred, normalize=True), 3)}", end='\n\n')
print(f"classification_report: {classification_report(y_test, pred)}", end='\n\n')

prediction: [1 1 2 1 3 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 4 1 3
 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 3 1 1 3 1 1 1
 2 1 2 3 1 1 4 1 3 1 1 1 1 1 1 3 1 1 1 1 1 3 1 1 1 3 1 3 1 1 1 1 1 1 1 1 1
 1 3 1 1 1 1 1 1 1 4 1 1 1 1 1 4 2 1 1 1 1 1 1 2 1 1 1 1 1 1 4 1 1 1 1 1 1
 4 1 1 1 1 1 1 1 4 1 1 2 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 4 1 1 1 1 1 1 1
 1 1 1 1 3 2 4 1 1 2 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 4 1 1 4 1 1
 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 4 1 2 3 1 1 1 4 1 4 1 1 1 4 1
 1 1 1 1 1 1 2 4 3 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 4 1 1
 1 1 1 1 1 1 1 1 1 4 1 1 2 1 1 1 1 1 1 1 1 2 1 1 3 1 1 1 1 2 1 3 1 2 1 4 1
 1 1 1 1 1 2 1 1 1 1 2 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1
 1 1 1 3 1 3 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 4 1 2 3 1 1
 1 1 4 1 4 4 

## 6. Подбор гиперпараметров

In [8]:
parameters: dict[str, Union[tuple[str, str], list[int]]] = {
    'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
    'gamma': ('scale', 'auto'),
    'C': [1, 10]
}

grid: GridSearchCV = GridSearchCV(
    SVC(),
    parameters,
    cv=5
)
results: Union[GridSearchCV, Any] = grid.fit(x_train, y_train)
print(results)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 10], 'gamma': ('scale', 'auto'),
                         'kernel': ('linear', 'poly', 'rbf', 'sigmoid')})


## 7. Заново обучим модель с подобранными гиперпараметрами

In [9]:
print(f"params: {results.best_params_}", end='\n\n')
print(f"score: {round(results.best_score_, 3)}", end='\n\n')
print(f"prediction: {results.predict(x_test)}", end='\n\n')

pred = results.predict(x_test)
print(f"accuracy_score: {round(accuracy_score(y_test, pred, normalize=True), 3)}")

params: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

score: 0.935

prediction: [1 1 2 1 3 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 4 1 3
 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 3 1 1 3 1 1 1
 4 1 2 3 1 1 4 1 3 1 1 1 1 1 1 3 1 1 1 1 1 3 1 1 1 3 1 3 1 1 1 1 1 4 1 1 1
 1 3 1 1 1 1 1 1 1 4 1 1 1 1 1 4 4 1 1 1 1 1 1 2 1 1 1 1 1 1 4 1 1 1 1 1 1
 4 1 1 1 1 1 1 1 4 1 1 2 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 4 1 1 1 1 1 1 1
 1 1 1 1 3 2 4 1 1 2 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 4 1 1 4 1 1
 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 4 1 2 3 1 1 1 4 1 4 1 1 1 4 1
 1 1 1 1 1 1 2 4 3 1 2 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 4 1 1
 1 1 1 1 1 1 1 1 1 4 1 1 2 1 1 1 1 1 1 1 1 2 1 1 3 1 1 1 1 2 1 3 1 2 1 4 1
 1 1 4 1 1 2 1 1 1 1 4 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1
 1 1 1 3 1 3 1 1 1 1

# Ответы на контрольные вопросы

## 1. Опишите этапы построения алгоритма классификации SVC. В чем его отличие от других линейных алгоритмов?

SVM – (support vector machine) – «машина опорных векторов» считается одним из лучших методов классификации. Он обладает несколькими замечательными свойствами.

Во-первых, обучение SVM сводится к задаче квадратичного программирования, имеющей единственное решение, которое вычисляется достаточно эффективно даже на выборках в сотни тысяч объектов.

Во-вторых, решение обладает свойством разреженности: положение оптимальной разделяющей гиперплоскости зависит лишь от небольшой доли обучающих объектов. Они и называются опорными векторами; остальные объекты фактически не задействуются.

Наконец, с помощью изящного математического приёма введения функции ядра метод обобщается на случай нелинейных разделяющих поверхностей. Вопрос о выборе ядра, оптимального для данной прикладной задачи, до сих пор остаётся открытой теоретической проблемой.

## 2. Какие гиперпараметры влияют на качество работы классификационной модели опорных векторов SVC? Каким образом их можно настроить?

Gamma является параметром ядра RBF и может рассматриваться как «разброс» ядра и, следовательно, область принятия решения. Когда gamma маленькая, «кривая» границы принятия решения очень низкая, и, следовательно, область принятия решения очень широкая. Когда gamma большая, «кривая» границы принятия решения высока, что создает островки границ принятия решения вокруг точек данных.

C является параметром обучаемого SVC и является штрафом за неправильную классификацию точки данных. Когда C мала, классификатор работает с неверно классифицированными точками данных (высокая систематическая ошибка, низкая дисперсия). Когда C большая, классификатор подвергается серьезным штрафам за неверно классифицированные данные и поэтому изгибается назад, чтобы избежать неверно классифицированных точек данных (низкая систематическая ошибка, высокая дисперсия).

Увеличение gamma приводит к сужению колоколообразной кривой (см. график слева на рис.), в результате чего сфера влияния каждого образца уменьшается: граница решений становится более неравномерной, извивающейся вблизи индивидуальных образцов. И наоборот, небольшое значение gamma делает колоколообразную кривую шире, поэтому образцы имеют большую сферу влияния, а граница решений оказывается более гладкой. Таким образом, γ действует аналогично гиперпараметру регуляризации: если ваша модель переобучается, тогда вы должны уменьшить значение gamma, а если недообучается — то увеличить его (подобно г иперпараметру C).

Эти параметры можно настроить методом перебора или с применением GridSearchCV, который принимает на вход набор параметров, образуя тем самым сетку. После чего происходит подбор параметров, соответствующих наилучшему результату, с помощью перекрёстного поиска по сетке.

## 3. Опишите параметры лучшей модели, полученной в лабораторной работе.

Параметры лучшей модели:

*	gamma - коэффициент ядра для 'rbf', 'poly' и 'sigmoid', по умолчанию в нашей модели используется «авто», т.е. использует 1 / n_features.
*	используемое ядро RFB – это функция близости, используемая в методике решения нелинейных задач, применяется для подсчитывания добавляемых признаков: $$\phi_\gamma(x,l)=exp(-\gamma||x-l||^2)$$
*	C - параметр SVC является штрафом за неправильную классификацию точки данных, в нашем случае он равен 10.
